# WRRC - Pulling meta data from PDFs

In [41]:
#install whatever libraries I've used which you don't have
#any errors you're getting are probably becuase you don't have the correct modules installed
#these are probably not all of them
#you can uncomment the ones below and run this cell to install them
#!pip install yake
#!pip install bs4
#!pip install pdftotext

In [33]:
from bs4 import BeautifulSoup
import requests, lxml, os, json
import pdftotext
import re
import yake

# Clean pdf text

In [34]:
with open("./morePdfs/shuler.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)
    
pdf_no_newlines = pdf[0].replace("\n", " ")

pdf_no_newlines += pdf[1].replace("\n", " ")

pdf_no_newlines += pdf[2].replace("\n", " ")

pdf_w_spaces = re.sub(r"[^a-zA-Z0-9 ]", "", pdf_no_newlines)

pdf_cropped = pdf_w_spaces[:4000]

pdf_split = pdf_cropped.split()

pdf_final = " ".join(pdf_split)

pdf_final

'hydrology Article Assessment of Terrigenous Nutrient Loading to Coastal Ecosystems along a Human LandUse Gradient Tutuila American Samoa Christopher K Shuler 12 Daniel W Amato 23 Veronica Gibson 3 Lydia Baker 4 Ashley N Olguin 5 Henrietta Dulai 12 Celia M Smith 3 and Rosanna A Alegado 567 1 Department of Earth Sciences University of Hawaii at Manoa Honolulu HI 96822 USA hdulaiovhawaiiedu 2 Water Resources Research Center University of Hawaii at Manoa Honolulu HI 96822 USA dwamatohawaiiedu 3 Botany University of Hawaii at Manoa Honolulu HI 96822 USA vgibsonhawaiiedu VG celiahawaiiedu CMS 4 Department of Microbiology Cornell University Ithaca NY 14850 USA lb693cornelledu 5 Daniel K Inouye Center for Microbial Oceanography Research and Education University of at Hawaii Manoa Honolulu HI 96822 USA ashleyolguingmailcom ANO ralegadohawaiiedu RAA 6 Department of Oceanography University of Hawaii at Manoa Honolulu HI 96822 USA 7 Sea Grant College Program University of Hawaii at Manoa Honolulu

# Pull keywords

In [35]:
kw_extractor = yake.KeywordExtractor()
text = pdf_final
language = "en"
max_ngram_size = 1
deduplication_threshold = 0.9
num_keywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=num_keywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
    print(kw)

('Honolulu', 0.003490874476577182)
('USA', 0.0036764024832059185)
('Manoa', 0.003734307882483832)
('Hawaii', 0.003981098312171747)
('University', 0.004079552855162242)
('Nutrient', 0.005499329497540566)
('Coastal', 0.007682947033702292)
('Department', 0.009648189116745448)
('Loading', 0.010331345394421414)
('American', 0.011791429655359613)
('Samoa', 0.011791429655359613)
('Water', 0.012924169032303217)
('hydrology', 0.01569571314164739)
('February', 0.01606546844104029)
('Ecosystems', 0.017617842902203928)
('Daniel', 0.018564749123754686)
('Anthropogenic', 0.019223580578650427)
('Terrigenous', 0.021130086966096757)
('Tutuila', 0.021130086966096757)
('Research', 0.021130086966096757)


# Google Scholar Scrape

In [36]:
headers = {
    'User-agent':
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

params = {
  #Three examples
  #"q": "SOURCE PARTITIONING OF ANTHROPOGENIC GROUNDWATER NITROGEN IN A MIXEDUSE LANDSCAPE TUTUILA AMERICAN SAMOA A THESIS SUBMITTED TO THE GRADUATE DIVISION OF THE UNIVERSITY OF HAWAII AT MNOA",
  #"q": pdf_no_newlines[:400]  
  "q": pdf_final[:400],  
  "hl": "en",
}

In [37]:
html = requests.get('https://scholar.google.com/scholar', headers=headers, params=params).text
soup = BeautifulSoup(html, 'lxml')

In [38]:
# Scrape just PDF links
for pdf_link in soup.select('.gs_or_ggsm a'):
    pdf_file_link = pdf_link['href']
    print(pdf_file_link)

https://www.mdpi.com/2306-5338/6/1/18/pdf


In [39]:
# JSON data will be collected here
data = []

# Container where all needed data is located
for result in soup.select('.gs_ri'):
    title = result.select_one('.gs_rt').text
    title_link = result.select_one('.gs_rt a')['href']
    publication_info = result.select_one('.gs_a').text
    snippet = result.select_one('.gs_rs').text
    cited_by = result.select_one('#gs_res_ccl_mid .gs_nph+ a')['href']
    related_articles = result.select_one('a:nth-child(4)')['href']
  
    try:
        all_article_versions = result.select_one('a~ a+ .gs_nph')['href']
    except:
        all_article_versions = None  
    
    data.append({
    'title': title,
    #'title_link': title_link,
    'publication_info': publication_info,
    'snippet': snippet,
    #'cited_by': f'https://scholar.google.com{cited_by}',
    #'related_articles': f'https://scholar.google.com{related_articles}',
    #'all_article_versions': f'https://scholar.google.com{all_article_versions}',
    })

# Results

In [40]:
print(data[0]['title'])
print()
print(data[0]['publication_info'])
print()
print(data[0]['snippet'])

Assessment of terrigenous nutrient loading to coastal ecosystems along a human land-use gradient, Tutuila, American Samoa

CK Shuler, DW Amato, V Gibson, L Baker, AN Olguin… - Hydrology, 2019 - mdpi.com

Anthropogenic nutrient loading is well recognized as a stressor to coastal ecosystem health. However, resource managers are often focused on addressing point source or surface water discharge, whereas the impact of submarine groundwater discharge (SGD) as a nutrient vector is often unappreciated. This study examines connections between land use and nutrient loading through comparison of four watersheds and embayments spanning a gradient of human use impact on Tutuila, a high tropical oceanic island in American Samoa …
